In [1]:
# prompt: install folium and pandas

!pip install folium
!pip install pandas

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [4]:
import pandas as pd
import requests
from io import BytesIO

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

response = requests.get(URL)
spacex_df = pd.read_csv(BytesIO(response.content))

spacex_df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [11]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [12]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [14]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [15]:
from folium.features import DivIcon

# Initialize the map centered on NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch site in the DataFrame
for idx, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']

    # Add a circle for each launch site
    circle = folium.Circle(
        location=coordinate,
        radius=1000,
        color='#0000ff',  # blue circle
        fill=True,
        fill_color='#0000ff',
        fill_opacity=0.5
    ).add_child(folium.Popup(site_name))

    # Add a marker with a label for each launch site
    marker = folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#0000ff;"><b>{site_name}</b></div>'
        )
    )

    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


In [16]:
from folium.features import DivIcon

# Initialize the map centered on NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Define colors for success and failure
success_color = '#28a745'  # green
failure_color = '#dc3545'  # red

# Add launch sites with circles and labels as before
for idx, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']

    circle = folium.Circle(
        location=coordinate,
        radius=1000,
        color='#0000ff',  # blue for site
        fill=True,
        fill_color='#0000ff',
        fill_opacity=0.5
    ).add_child(folium.Popup(site_name))

    marker = folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#0000ff;"><b>{site_name}</b></div>'
        )
    )

    site_map.add_child(circle)
    site_map.add_child(marker)

# Now, add markers for each launch according to success or failure
for idx, row in spacex_df.iterrows():
    launch_coord = [row['Lat'], row['Long']]
    launch_site = row['Launch Site']
    outcome = row['class']

    # Choose color based on success or failure
    color = success_color if outcome == 1 else failure_color
    label = 'Success' if outcome == 1 else 'Failure'

    # Create a circle marker with popup indicating launch site and success/failure
    folium.CircleMarker(
        location=launch_coord,
        radius=6,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{launch_site} - {label}"
    ).add_to(site_map)

site_map


In [17]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [18]:
marker_cluster = MarkerCluster()


In [24]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


In [23]:
from folium.plugins import MarkerCluster
import folium

marker_cluster = MarkerCluster()

for index, record in spacex_df.iterrows():
    # Create a Marker with the launch site coordinates
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=record['marker_color']),
        popup=(
            f"Launch Site: {record['Launch Site']}<br>"
            f"Launch Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
        )
    )
    marker_cluster.add_child(marker)

# Add the cluster of markers to the site_map
site_map.add_child(marker_cluster)

# Display the map
site_map


In [25]:
# First, compute success and total launch counts per site
summary_df = spacex_df.groupby('Launch Site')['class'].agg(['sum', 'count']).reset_index()
summary_df.columns = ['Launch Site', 'Successes', 'Total']

# Merge back to launch_sites_df
launch_sites_df = launch_sites_df.merge(summary_df, on='Launch Site')

# Re-create the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add each launch site as a circle and a marker with success summary
for index, site in launch_sites_df.iterrows():
    coord = [site['Lat'], site['Long']]
    popup_text = (f"{site['Launch Site']}<br>"
                  f"Successes: {site['Successes']} / {site['Total']}<br>"
                  f"Success Rate: {site['Successes'] / site['Total']:.2%}")

    folium.Circle(coord, radius=1000, color='#000000', fill=True).add_child(
        folium.Popup(popup_text)
    ).add_to(site_map)

    folium.Marker(coord, icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{site["Launch Site"]}</b></div>',
    )).add_to(site_map)

# Add marker cluster again (assuming it’s already created earlier)
site_map.add_child(marker_cluster)
site_map


In [26]:
from geopy.distance import geodesic

# Function to calculate distance between two lat/lon points
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).km


In [27]:
# NASA JSC coordinate
nasa_jsc_coord = [29.559684888503615, -95.0830971930759]

# Create a new column for the distance to NASA JSC
launch_sites_df['Distance_to_NASA_JSC_km'] = launch_sites_df.apply(
    lambda row: calculate_distance([row['Lat'], row['Long']], nasa_jsc_coord),
    axis=1
)

launch_sites_df[['Launch Site', 'Distance_to_NASA_JSC_km']]


,Launch Site,Distance_to_NASA_JSC_km
0,CCAFS LC-40,1415.986029
1,CCAFS SLC-40,1416.024225
2,KSC LC-39A,1409.079675
3,VAFB SLC-4E,2467.223266


In [28]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [29]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [30]:
# Launch site coordinates
launch_lat = 28.5623
launch_lon = -80.5774

# Coastline coordinates
coast_lat = 28.56367
coast_lon = -80.57163

# Calculate distance
distance_coastline = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)

# Add marker to coastline
distance_marker = folium.Marker(
    [coast_lat, coast_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add marker to the map
site_map.add_child(distance_marker)

# Optionally add a line between the two points for visualization
line = folium.PolyLine(locations=[[launch_lat, launch_lon], [coast_lat, coast_lon]], color='blue', weight=2.5)
site_map.add_child(line)

site_map


In [31]:
# Coordinates
launch_site_coord = [28.5623, -80.5774]
coastline_coord = [28.56367, -80.57163]

# Create a PolyLine
lines = folium.PolyLine(locations=[launch_site_coord, coastline_coord], weight=2, color='blue')

# Add the PolyLine to the map
site_map.add_child(lines)

# Show the map
site_map


In [32]:
# Example coordinates (replace with actual)
launch_site_coord = [28.5623, -80.5774]    # Launch site
poi_coord = [28.5630, -80.5700]            # Point of Interest (e.g., closest city)

# Calculate distance in km
distance_km = calculate_distance(
    launch_site_coord[0], launch_site_coord[1],
    poi_coord[0], poi_coord[1]
)

# Create a marker at the point of interest with a label showing the distance
distance_marker = folium.Marker(
    location=poi_coord,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color: blue;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
)

# Draw a line between the launch site and the point of interest
line = folium.PolyLine(
    locations=[launch_site_coord, poi_coord],
    weight=2,
    color='green'
)

# Add marker and line to the map
site_map.add_child(distance_marker)
site_map.add_child(line)

# Display map
site_map
